# Catanomics - Computer Vision and pre-processing
The goal of this notebook is to read in an image of a catan board without number or settlements, and draw the board (ports not included) over the image

In [11]:
# Import necessary libraries
import cv2
import numpy as np

In [12]:
# Show image function for later use
def showImage(img, name=None):
    if not name:
        cv2.imshow("Image display", img)
    else:
        cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [13]:
# Save image function
def saveImage(filename, img, dir):
    # Get full path
    full_path = f"{dir}/{filename}"
    cv2.imwrite(full_path, img)
    print(f"Image saved to {full_path}")

## 1. Read in the image, detect the border, and find the max/min y point of the permiter

In [14]:
# Read in an image
image = cv2.imread('../images/v3/board01.jpg')
# The input imgs are too big, so reduce to 25%
image = cv2.resize(image, (0,0), fx=.25, fy=.25)
showImage(image)

In [15]:
# Convert the image to hsv
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

In [16]:
# Define color bounds for the perimiter of the board
lower_blue = np.array([100, 150, 0])
upper_blue = np.array([140, 255, 255])

# Create a binary mask where the blue regions are white, and everything else is black
mask = cv2.inRange(hsv_image, lower_blue, upper_blue)

In [17]:
# Bitwise AND to keep the blue parts of the image
blue_regions = cv2.bitwise_and(image, image, mask=mask)

In [18]:
# Perform canny edge detection on the blue_regions

# Convert to grayscale
gray_blue_region = cv2.cvtColor(blue_regions, cv2.COLOR_BGR2GRAY)
# Apply Gaussian Blur
blurred_blue_regions = cv2.GaussianBlur(gray_blue_region, (5,5), 0)
# Canny edge detection
edges = cv2.Canny(blurred_blue_regions, 50, 150)
# Show the result
showImage(edges, "Edges in blue regions")

## 2. Perform Hough Transform on the images

In [19]:
# Perform Hough Transform using `HoughLines`
lines = []
thresh = 200
while len(lines) < 6:
    lines = cv2.HoughLines(edges, 1, np.pi / 180, thresh)
    print(f"Trying threshold {thresh} | Got {len(lines)} lines.")
    if len(lines) >= 6:
        break
    else:
        thresh -= 5

print(f"Results: {thresh} threshold | {len(lines)} lines")

Trying threshold 200 | Got 3 lines.
Trying threshold 195 | Got 3 lines.
Trying threshold 190 | Got 3 lines.
Trying threshold 185 | Got 3 lines.
Trying threshold 180 | Got 3 lines.
Trying threshold 175 | Got 3 lines.
Trying threshold 170 | Got 3 lines.
Trying threshold 165 | Got 3 lines.
Trying threshold 160 | Got 3 lines.
Trying threshold 155 | Got 4 lines.
Trying threshold 150 | Got 4 lines.
Trying threshold 145 | Got 4 lines.
Trying threshold 140 | Got 5 lines.
Trying threshold 135 | Got 5 lines.
Trying threshold 130 | Got 5 lines.
Trying threshold 125 | Got 6 lines.
Results: 125 threshold | 6 lines


In [20]:
# Draw lines on the image
for line in lines:
    rho, theta = line[0]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(image, (x1, y1), (x2, y2), (0, 0, 255), 2)
showImage(image)
saveImage("perimiter00.jpg", image, "../images/perimiter/v1")

Image saved to ../images/perimiter/v1/perimiter00.jpg
